In [16]:
#import everything upwards from home dir
from __future__ import absolute_import
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from configparser import ConfigParser
import psycopg2
import argparse
import six
import sys
import google.ads.google_ads.client
import pandas as pd
import re
import numpy as np
import time

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

def get_report(df_conf_req, view_id, key_file_location, scopes):
    
    
    # Initializes an Analytics Reporting API V4 service object.
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
            key_file_location, scopes)

    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    
    # define empty pandas dataframe
    df_response = pd.DataFrame()
    
    # create lists for metrics and dimensions
    dim_lst = []
    met_lst = []
    start_date = df_conf_req.iat[0,2]
    end_date = df_conf_req.iat[1,2]
    for index, req in df_conf_req.iterrows():
        if not 'nan' in str(req['dimensions']):
            dim_lst.append(dict({'name':req['dimensions']}))
        if not 'nan' in str(req['metrics']):
            met_lst.append(dict({'expression':req['metrics']}))  
    
    dim_met_lst = dim_lst +  met_lst
    
    met_batches = list()
    
    # split metric list into batches, since a single API call can have a max of 10 metrics
    i = 0
    while True:
        met_batch = list()
        stop = 0
        for index, met in enumerate(met_lst):
            if len(met_batch) < 10 and i < len(met_lst):
                res = met_batch.append(met_lst[i])
                i=i+1
        if len(met_batch) != 0:
            met_batches.append(met_batch)
        if i == len(met_lst):
            x = True
            break 

    # create empty dataframe for response segment
    df_res_part = pd.DataFrame()
    
    # iterate over metric batches
    for index, batch in enumerate(met_batches):
        
        # define request body
        body={
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': batch,
            'dimensions': dim_lst,
            
            'pageSize': 10000,
            'includeEmptyRows': True

        }]
    }    
        # make the call to Google Analytics API
        response = analytics.reports().batchGet(body=body).execute()
        
        # deconstruct JSON response
        for report in response.get('reports', []):
            columnHeader = report.get('columnHeader', {})
            dimensionHeaders = columnHeader.get('dimensions', [])
            metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

            # iterate over rows
            for data_row in report.get('data', {}).get('rows', []):
                dimensions = data_row.get('dimensions', [])
                dateRangeValues = data_row.get('metrics', [])
                var_dict = {}
                
                #iterate over dimensions
                for header, dimension in zip(dimensionHeaders, dimensions):
                    if header not in df_res_part:
                        df_res_part[header] = pd.Series()
                        df_res_part.astype({header: 'object'}).dtypes
                    var_dict.update({header : str(dimension)})
                    
                # iterate over metrics
                for i, values in enumerate(dateRangeValues):
                    for metricHeader, value in zip(metricHeaders, values.get('values')):
                        if metricHeader.get('name') not in df_res_part.columns:
                            df_res_part[metricHeader.get('name')] = pd.Series()
                        var_dict.update({metricHeader.get('name') : value})
                df_res_part = df_res_part.append(var_dict, ignore_index=True)
        row_count_part = len(df_res_part.index)
    # if iteration is first and main dataframe is empty assign current response segment
    if df_response.empty:
        df_response = df_res_part
    # else do a left join and combine the two
    else:
        df_response = pd.merge(df_response, df_res_part,  how='left', left_on=['ga:campaign', 'ga:adcontent', 'ga:channelGrouping', 'ga:keyword', 'ga:date'], right_on = ['ga:campaign', 'ga:adcontent', 'ga:channelGrouping', 'ga:keyword', 'ga:date'])
    row_count_full = len(df_response.index)
    
    return df_response, dim_met_lst

def db_config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

def postgre_write(df_response, dim_lst, view_id):
    #Connect to the PostgreSQL database server
    conn = None
    # read connection parameters
    params = db_config()
    
    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**params)
 
    try:
        # create a cursor
        cur = conn.cursor()

       # execute db write statements
        print('PostgreSQL database version:')
        # *tread lightly* cur.execute("DROP TABLE google_analytics_temp;")
        cur.execute("CREATE TABLE IF NOT EXISTS google_analytics_temp("
                       "ga_viewID bigint, "
                       "ga_sourceMedium VARCHAR(200), "
                       "ga_date DATE, "
                       "ga_campaign VARCHAR(150), "
                       "ga_adcontent VARCHAR(150), "
                       "ga_channelGrouping VARCHAR(150), "
                       "ga_keyword VARCHAR(150), "
                       "CONSTRAINT table_analytics_pk PRIMARY KEY (ga_viewID, ga_sourceMedium, ga_date, ga_campaign, ga_adcontent, ga_channelGrouping, ga_keyword));")
        conn.commit()    
        
        # iterate over rows in response
        for i, row in df_response.iterrows():
            ins_query_dim ='ga_viewID, '
            ins_query_val = f"{view_id}, "
            upd_query = ''
            col_dtype = ''
            
            # iterate over parameters in every row
            for idx, dim in enumerate(list(row.index)):
                db_dim = dim
                val = str(row[dim])
                
                # replace DB illegal char with info string
                uns_char_lst = ["'", ":"]
                for char in uns_char_lst:
                    if char in str(row[dim]):
                        val = val.replace(char, "_uns_char_fnd_ascii_dec_expr_" + str(ord(char)))
                    if char in db_dim:
                        db_dim = db_dim.replace(char, '_')
                # conditionals for variable type assignment
                if str(row[dim]) != 'nan':
                    if str(db_dim) == 'ga_sessions':
                        col_dtype = 'bigint'
                        cur.execute(f"ALTER TABLE google_analytics_temp "
                                    f"ADD COLUMN IF NOT EXISTS {db_dim} {col_dtype};")
                        ins_query_dim = ins_query_dim + db_dim + ','
                        ins_query_val = ins_query_val + " '" + val + "',"
                        upd_query = upd_query + db_dim + ' = ' + val + ', '
                        
                    elif 'goal' in str(db_dim) or 'transaction' in str(db_dim):
                        col_dtype = 'real'
                        cur.execute(f"ALTER TABLE google_analytics_temp "
                                    f"ADD COLUMN IF NOT EXISTS {db_dim} {col_dtype};")
                        ins_query_dim = ins_query_dim + db_dim + ','
                        ins_query_val = ins_query_val + " '" + val + "',"
                        upd_query = upd_query + db_dim + ' = ' + val + ', '
                    else:
                        col_dtype = 'varchar (150)'
                        cur.execute(f"ALTER TABLE google_analytics_temp "
                                    f"ADD COLUMN IF NOT EXISTS {db_dim} {col_dtype};")
                        ins_query_dim = ins_query_dim + db_dim + ','
                        ins_query_val = ins_query_val + " '" + val + "',"
            ins_query_dim = ins_query_dim[:-1]
            ins_query_val = ins_query_val[:-1]
            upd_query = upd_query[:-2]

            # insert into db or if row exists update all metrics
            cur.execute(f"INSERT INTO google_analytics_temp ({ins_query_dim}) "
                        f"VALUES ({ins_query_val}) "
                        f"ON CONFLICT (ga_viewID, ga_sourceMedium, ga_date, ga_campaign, ga_adcontent, ga_channelGrouping, ga_keyword) " 
                            f"DO "
                                f"UPDATE "
                                f"SET {upd_query}; ")
            conn.commit()
       
       # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    print('Database connection closed.')

if __name__ == '__main__':
    
    # read configuration from excel
    df_conf_base = pd.read_excel('google_analytics_conf_1.xlsx', sheet_name='base', header=0)
    df_conf_req = pd.read_excel('google_analytics_conf_1.xlsx', sheet_name='parameters', header=0) 
    
    scopes = ['https://www.googleapis.com/auth/analytics.readonly']
    key_file_location = 'client_secrets.json'
    
    # iterate over view IDs
    for index, row in df_conf_base.iterrows():
        view_id = str(row['view_id'])
        #view_id = str(df_conf_base.iat[0,0])

        # call defined methods
        google_analytics_response = get_report(df_conf_req, view_id, key_file_location, scopes)
        df_response = google_analytics_response[0]
        dim_met_lst = google_analytics_response[1]

        db_config(filename = 'database.ini')
        postgre_write(df_response, dim_met_lst, view_id)
        
        

Connecting to the PostgreSQL database...
PostgreSQL database version:


KeyboardInterrupt: 